# Lab 06: Gene Expression

---
author: Your Name Here
date: October 18, 2024
embed-resources: true
---

## Introduction

## Methods

In [15]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge, Lasso

### Data

In [2]:
# load data
cancer_train = pd.read_parquet(
    "https://cs307.org/lab-06/data/cancer-train.parquet",
)
cancer_test = pd.read_parquet(
    "https://cs307.org/lab-06/data/cancer-test.parquet",
)
cancer_train

,cancer,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_1990,gene_1991,gene_1992,gene_1993,gene_1994,gene_1995,gene_1996,gene_1997,gene_1998,gene_1999
0,BRCA,0.0,3.149861,1.913454,5.562355,9.638586,0.0,4.302421,0.511670,0.000000,...,7.300691,8.383307,1.187198,0.000000,7.350471,0.0,4.634587,7.082415,9.727447,1.187198
1,LUAD,0.0,6.237034,5.043235,6.297397,10.391415,0.0,7.669941,0.913033,0.000000,...,6.322446,7.815595,13.809095,0.913033,7.651052,0.0,7.476074,4.733739,8.510863,0.000000
2,BRCA,0.0,3.856896,2.394981,6.758277,9.585513,0.0,7.409009,1.242023,0.000000,...,6.845515,9.194823,5.667696,0.000000,7.748253,0.0,5.567421,5.203158,7.364879,0.000000
3,PRAD,0.0,4.279924,3.606963,5.706613,9.716581,0.0,8.244226,0.402613,0.000000,...,6.598611,8.199118,5.024218,0.000000,7.607907,0.0,5.705281,6.278007,9.725383,0.000000
4,BRCA,0.0,3.359788,4.199986,6.144766,9.141834,0.0,9.014135,1.061776,0.626486,...,7.062651,9.670708,4.243707,0.000000,8.156811,0.0,7.363487,5.384844,8.703443,1.894876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,KIRC,0.0,3.069737,3.623200,6.744955,9.591219,0.0,7.254254,0.000000,0.000000,...,7.333665,8.365334,5.481444,0.000000,7.307674,0.0,5.723447,5.401791,9.067881,0.000000
124,BRCA,0.0,3.534497,3.064866,6.638882,10.010206,0.0,7.899387,0.000000,0.000000,...,7.080615,9.968912,10.501041,0.000000,8.866052,0.0,5.276806,4.941073,8.548240,0.000000
125,BRCA,0.0,4.087463,3.786596,6.385845,9.544964,0.0,8.062856,0.000000,0.000000,...,5.842979,9.931033,10.234817,0.000000,8.384568,0.0,7.033423,5.749534,8.132371,0.000000
126,LUAD,0.0,3.272889,4.529234,7.134909,9.504362,0.0,5.668893,0.000000,0.000000,...,7.300966,8.040350,11.972872,0.000000,7.592607,0.0,5.026256,6.137704,8.780947,1.167936


In [3]:
# summary statistics
# create X and y for train
X_train = cancer_train.drop(columns=["cancer"])
y_train = cancer_train["cancer"]

# create X and y for test
X_test = cancer_test.drop(columns=["cancer"])
y_test = cancer_test["cancer"]

In [ ]:
# visualizations

### Models

In [7]:
numeric_features = X_train.select_dtypes(include=["float64", "int64"]).columns
categorical_features = ["cancer"]

In [20]:
# process data for ML
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
    ]
)


# combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# create pipeline, preprocess then classify
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("Classifier", LogisticRegression()),
    ]
)

In [21]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  Index(['gene_0', 'gene_1', 'gene_2', 'gene_3', 'gene_4', 'gene_5', 'gene_6',
       'gene_7', 'gene_8', 'gene_9',
       ...
       'gene_1990', 'gene_1991', 'gene_1992', 'gene_1993', 'gene_1994',
       'gene_1995', 'gene_1996', 'gene_1997', 'gene_1998', 'gene_1999'],
      dtype='object', length=2000))])),
                ('Classifier', LogisticRegression())])

In [26]:
param_grid = [
    {
        "Classifier": [DummyClassifier()],
    },
    {
        "Classifier": [RandomForestClassifier()],
        "Classifier__max_features": [1, 3, 10],
        "Classifier__n_estimators": [25, 100],
        "Classifier__max_samples": [10, None],
    },
    {
        "Classifier": [LogisticRegression()],
        "Classifier__penalty": ["l1", "l2"],
        "Classifier__solver": ["liblinear"],
        "Classifier__C": 1 / (10.0 ** np.arange(-4, 3))
    },
]

In [27]:
mod = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="accuracy",
    verbose=1,
)
mod.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer())]),
                                                                         Index(['gene_0', 'gene_1', 'gene_2', 'gene_3', 'gene_4', 'gene_5', 'gene_6',
       'gene_7', 'gene_8', 'gene_9',
       ...
       'gene_1990', 'gene_1991', 'gene_1992', 'gene_1993', 'gene_1994',
       'gene_1995', 'gene_1996', 'gene_1997', 'gene_199...
                         {'Classifier': [RandomForestClassifier()],
                          'Classifier__max_features': [1, 3, 10],
                          'Classifier__max_samples': [10, None],
                          'Classifier__n_estimators': [25, 100]},
                         {'Classifier': [LogisticRegression()],
                          'Classifier__C': array([1.e+04, 1.e+03, 1.e+02, 1.e+01, 1.e+00, 1.e-01, 1.e-02]),
                          'Classifier__penalty': ['l1', 'l2'],
                          'Classifier__solver': ['liblinear']}],
             scoring='accuracy', verbose=1)

In [28]:
print(f"Best parameters: {mod.best_params_}")
print(f"Best cross-validation accuracy: {mod.best_score_}")

Best parameters: {'Classifier': LogisticRegression(), 'Classifier__C': 10000.0, 'Classifier__penalty': 'l1', 'Classifier__solver': 'liblinear'}
Best cross-validation accuracy: 1.0


In [31]:
y_pred = mod.predict(X_test)
print(f"Test Accuracy: {mod.score(X_test, y_test):.2f}")

Test Accuracy: 1.00


In [32]:
from joblib import dump
dump(mod, "gene-expression.joblib")

['gene-expression.joblib']

In [ ]:
# train models

## Results

In [ ]:
# report model metrics

## Discussion

### Conclusion